In [1]:
import numpy as np
import mne
from mne.datasets.sleep_physionet.age import fetch_data
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np


# Annotation descriptions to event ID mapping
annotation_desc_2_event_id = {
    "Sleep stage W": 1,
    "Sleep stage 1": 2,
    "Sleep stage 2": 3,
    "Sleep stage 3": 4,
    "Sleep stage 4": 4,
    "Sleep stage R": 5,
}
"""
The function is useful in EEG analysis for feature engineering, where statistical characteristics
of the power spectral distribution across different frequency bands may correlate with various 
neurological conditions or states. The output is used as input for machine learning models 
in EEG-based studies.
"""
def enhanced_eeg_power_band(epochs):
    # Frequency bands for common ranges
    FREQ_BANDS = {
        "delta": [0.5, 4.5],
        "theta": [4.5, 8.5],
        "alpha": [8.5, 11.5],
        "sigma": [11.5, 15.5],
        "beta": [15.5, 30],
    }
   # Compute the power spectral density for EEG signals
   # compute_psd returns a spectrum object from which data
   # (psds) and corresponding frequencies (freqs) are 
   # extracted using spectrum.get_data
    
    spectrum = epochs.compute_psd(picks='eeg', fmin=0.5, fmax=30.0)
    psds, freqs = spectrum.get_data(return_freqs=True)
    
   # This normalization is important for removing variability due to different signal strengths and 
   # focusing on the distribution shape across frequencies. 
    psds /= np.sum(psds, axis=-1, keepdims=True)  # Normalization
    """
    The below code loops over each frequency band defined in FREQ_BANDS. For each band:
    psds_band extracts the PSD values that fall within the current band's frequency range.
    mean_psds_band calculates the mean of these PSD values along the frequency axis.
    skew_psds_band computes the skewness of these PSD values, which is a measure of the asymmetry of the distribution.
    kurtosis_psds_band computes the kurtosis of these PSD values, indicating the tailedness of the distribution.
    These three statistical measures are then concatenated along the feature axis (axis=1), forming a feature vector for the current band.
    This feature vector is appended to the list X.

    """

    X = []
    # Extract mean, skew, and kurtosis for each band
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)]
        mean_psds_band = psds_band.mean(axis=-1)
        skew_psds_band = skew(psds_band, axis=-1)
        kurtosis_psds_band = kurtosis(psds_band, axis=-1)
        features = np.concatenate([mean_psds_band, skew_psds_band, kurtosis_psds_band], axis=1)
        X.append(features.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

# Load data
def load_and_preprocess_data(x):
    subjects = range(int(x))
    all_features = []
    all_labels = []
    epochs_all =[]
    for subject in subjects:
        files = fetch_data(subjects=[subject], recording=[1])
        raw = mne.io.read_raw_edf(files[0][0], preload=True)
        raw.filter(0.5, 30, fir_design='firwin')
        annots = mne.read_annotations(files[0][1])
        raw.set_annotations(annots)
        events, event_id  = mne.events_from_annotations(raw, event_id=annotation_desc_2_event_id, chunk_duration=30.0)
        tmax = 30.0 - 1.0 / raw.info["sfreq"]  # tmax in included
        epochs =mne.Epochs(raw, events, event_id=annotation_desc_2_event_id, tmin=0., tmax=tmax, baseline=None)
        #epochs = enhanced_eeg_power_band(epo)
        # epochs_all.append(epo)
        labels = epochs.events[:, -1] - 1  # Adjust labels to be zero-indexed
        y = epochs.get_data()
        all_features.append(y)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels) - 1  # Adjust labels to be zero-indexed



ModuleNotFoundError: No module named 'mne'

In [3]:
X, y = load_and_preprocess_data(10)


Using default location ~/mne_data for PHYSIONET_SLEEP...
Extracting EDF parameters from C:\Users\91908\mne_data\physionet-sleep-data\SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\91908\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,
C:\Users\91908\AppData\Local\Temp\ipykernel_48572\2006941326.py:81: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(files[0][0], preload=True)
C:\Users\91908\AppData\Local\Temp\ipykernel_48572\2006941326.py:81: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(files[0][0], preload=True)
C:\Users\91908\AppData\Local\Temp\ipykernel_48572\2006941326.py:81: RuntimeWarning: Highpass cutoff frequency 16.0 is greater than lowpass cutoff frequency 0.7, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(files[0][0], preload=True)


Reading 0 ... 7949999  =      0.000 ... 79499.990 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 661 samples (6.610 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    2.4s finished
C:\Users\91908\AppData\Local\Temp\ipykernel_48572\2006941326.py:84: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(annots)


Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
2650 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 2650 events and 3000 original time points ...
0 bad epochs dropped
Using default location ~/mne_data for PHYSIONET_SLEEP...
Extracting EDF parameters from C:\Users\91908\mne_data\physionet-sleep-data\SC4011E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8405999  =      0.000 ... 84059.990 secs...


C:\Users\91908\AppData\Local\Temp\ipykernel_48572\2006941326.py:81: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(files[0][0], preload=True)
C:\Users\91908\AppData\Local\Temp\ipykernel_48572\2006941326.py:81: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(files[0][0], preload=True)
C:\Users\91908\AppData\Local\Temp\ipykernel_48572\2006941326.py:81: RuntimeWarning: Highpass cutoff frequency 16.0 is greater than lowpass cutoff frequency 0.7, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(files[0][0], preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 661 samples (6.610 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    2.9s finished
C:\Users\91908\AppData\Local\Temp\ipykernel_48572\2006941326.py:84: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(annots)


Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
2802 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 2802 events and 3000 original time points ...
0 bad epochs dropped


In [4]:

# pip install xgboost
X.shape

(5452, 7, 3000)

In [32]:
np.unique(y_train_res)

array([0, 1, 2, 3, 4])

In [6]:
y = y+1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)
# X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)



In [7]:
X_train_scaled.shape

(4361, 7, 3000)

In [68]:
# X_train_reshaped.shape

In [8]:
import numpy as np
from imblearn.over_sampling import SMOTE

# Assuming X_train is your features array of shape (1051, 7, 3001)
# and y_train is your target array of shape (1051,)

# Reshape X_train from 3D to 2D
X_train_reshaped = X_train_scaled.reshape(X_train.shape[0], -1)  
# Initialize SMOTE
smote = SMOTE()

# Apply SMOTE
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_reshaped, y_train)

# Optional: Reshape X_train_resampled back to 3D if your model requires 3D input
# This step depends on whether your model accepts 3D data directly
X_train_resampled_3d = X_train_resampled.reshape(-1, X_train_scaled.shape[1], X_train_scaled.shape[2])

# Now, X_train_resampled_3d and y_train_resampled can be used for further modeling


In [9]:
X_train_res= X_train_resampled_3d
y_train_res = y_train_resampled

In [9]:
# X_train_res = X_train_res[..., np.newaxis] 
# X_test_scaled =  X_test_scaled[..., np.newaxis] 

In [10]:
X_train_res.shape

(15390, 7, 3000)

In [31]:
y_train_res.shape

(15390,)

In [49]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D, concatenate, Dropout, Conv1DTranspose, BatchNormalization, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Assuming y_train_res has been one-hot encoded
y_train_res_one_hot = to_categorical(y_train_res, num_classes=5)

def unet_model(input_shape=(3000, 7), num_classes=5):
    inputs = Input(input_shape)
    
    # Downsampling Path
    conv1 = Conv1D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv1D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling1D(pool_size=2)(conv1)
    
    conv2 = Conv1D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv1D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling1D(pool_size=2)(conv2)
    
    conv3 = Conv1D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv1D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling1D(pool_size=2)(conv3)
    
    # Bottleneck
    conv4 = Conv1D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv1D(512, 3, activation='relu', padding='same')(conv4)
    
    # Upsampling Path
    up7 = concatenate([UpSampling1D(size=2)(conv4), conv3], axis=-1)
    conv7 = Conv1D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv1D(256, 3, activation='relu', padding='same')(conv7)

    up8 = concatenate([UpSampling1D(size=2)(conv7), conv2], axis=-1)
    conv8 = Conv1D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv1D(128, 3, activation='relu', padding='same')(conv8)

    up9 = concatenate([UpSampling1D(size=2)(conv8), conv1], axis=-1)
    conv9 = Conv1D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv1D(64, 3, activation='relu', padding='same')(conv9)
    gap = GlobalAveragePooling1D()(conv9)  # use conv9 from the last upsample layer

    # Output Layer
    # conv10 = Conv1D(num_classes, 1, activation='softmax')(gap)
    output = Dense(num_classes, activation='softmax')(gap)

    model = Model(inputs=[inputs], outputs=[output])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Create and compile the model
model = unet_model()
# Configure Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Model summary
model.summary()


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20      │ (None, 3000, 7)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_127 (Conv1D) │ (None, 3000, 64)  │      1,408 │ input_layer_20[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_128 (Conv1D) │ (None, 3000, 64)  │     12,352 │ conv1d_127[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_31    │ (None, 1500, 64)  │          0 │ conv1d_128[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_129 (Conv1D) │ (None, 1500, 128) │     24,704 │ max_pooling1d_31… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_130 (Conv1D) │ (None, 1500, 128) │     49,280 │ conv1d_129[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_32    │ (None, 750, 128)  │          0 │ conv1d_130[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_131 (Conv1D) │ (None, 750, 256)  │     98,560 │ max_pooling1d_32… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_132 (Conv1D) │ (None, 750, 256)  │    196,864 │ conv1d_131[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_33    │ (None, 375, 256)  │          0 │ conv1d_132[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_133 (Conv1D) │ (None, 375, 512)  │    393,728 │ max_pooling1d_33… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_134 (Conv1D) │ (None, 375, 512)  │    786,944 │ conv1d_133[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling1d_22    │ (None, 750, 512)  │          0 │ conv1d_134[0][0]  │
│ (UpSampling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_27      │ (None, 750, 768)  │          0 │ up_sampling1d_22… │
│ (Concatenate)       │                   │            │ conv1d_132[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_135 (Conv1D) │ (None, 750, 256)  │    590,080 │ concatenate_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_136 (Conv1D) │ (None, 750, 256)  │    196,864 │ conv1d_135[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling1d_23    │ (None, 1500, 256) │          0 │ conv1d_136[0][0]  │
│ (UpSampling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_28      │ (None, 1500, 384) │          0 │ up_sampling1d_23… │
│ (Concatenate)       │                   │            │ conv1d_130[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_137 (Conv1D) │ (None, 1500, 128) │    147,584 │ concatenate_28[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_138 (Conv1D) │ (None, 1500, 128) │     49,280 │ conv1d_137[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling1d_24    │ (None, 3000, 128) │          0 │ conv1d_138[0][0]

 Total params: 2,597,253 (9.91 MB)

 Trainable params: 2,597,253 (9.91 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
# Example fit call; replace with actual data and parameters
X_train_res = np.transpose(X_train_res, (0, 2, 1))  # Reshape from (15390, 7, 3000) to (15390, 3000, 7)

In [53]:
X_train_res.shape

(15390, 3000, 7)

In [54]:
history = model.fit(X_train_res, y_train_res_one_hot, epochs=40, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 2469s 6s/step - accuracy: 0.6710 - loss: 0.7952 - val_accuracy: 0.0737 - val_loss: 3.8721
Epoch 2/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 9117s 24s/step - accuracy: 0.8989 - loss: 0.3053 - val_accuracy: 0.1170 - val_loss: 1.5908
Epoch 3/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 5338s 14s/step - accuracy: 0.9139 - loss: 0.2687 - val_accuracy: 0.1491 - val_loss: 1.9322
Epoch 4/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 10837s 28s/step - accuracy: 0.9206 - loss: 0.2381 - val_accuracy: 0.1014 - val_loss: 2.4167
Epoch 5/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 2312s 6s/step - accuracy: 0.9238 - loss: 0.2178 - val_accuracy: 0.0806 - val_loss: 4.1652
Epoch 6/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 2428s 6s/step - accuracy: 0.9248 - loss: 0.2311 - val_accuracy: 0.0900 - val_loss: 3.4954
Epoch 7/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 2909s 8s/step - accuracy: 0.9449 - loss: 0.1739 - val_accuracy: 0.2135 - val_loss: 2.0231
Epoch 8/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 2396s 6s/step - accuracy: 0.9408 - los

In [55]:
model.save('unet_model_final.h5')
print("Model saved as 'unet_model_final.h5'.")


Model saved as 'unet_model_final.h5'.


In [138]:
from tensorflow.keras.models import load_model

# Load the previously saved model
model_path = 'unet_model_final.h5'
loaded_model = load_model(model_path)
print("Model loaded successfully.")

Model loaded successfully.


In [139]:
# Assuming new_data is preprocessed and ready to be fed into the model
# Example new_data shape should match (None, 3000, 7)
X_test_scaled = np.transpose(X_test_scaled, (0, 2, 1))  
# Predict using the loaded model
predictions = loaded_model.predict(X_test_scaled)
print("Predictions completed.")

35/35 ━━━━━━━━━━━━━━━━━━━━ 32s 902ms/step
Predictions completed.


In [140]:
predictions

array([[9.9331927e-01, 6.6807773e-03, 8.7993736e-18, 1.2416628e-33,
        4.4214146e-10],
       [1.0000000e+00, 5.9626612e-26, 2.5663825e-30, 1.2950892e-22,
        4.2775293e-14],
       [1.0000000e+00, 1.4937697e-14, 9.5677550e-18, 5.5756592e-15,
        2.6702695e-08],
       ...,
       [1.0000000e+00, 3.7695178e-21, 3.4909223e-34, 1.0738566e-27,
        1.1620343e-17],
       [1.0000000e+00, 4.7553578e-32, 1.8777445e-27, 4.5029139e-26,
        2.8209639e-08],
       [0.0000000e+00, 8.3056931e-21, 1.7761324e-01, 8.2238674e-01,
        6.4128062e-21]], dtype=float32)

In [141]:
class_predictions = np.argmax(predictions, axis=1)

In [142]:
acc = accuracy_score(y_test, class_predictions)
print("Accuracy score: {}".format(acc))

Accuracy score: 0.9495875343721356


In [143]:
final_accuracy = accuracy_score(y_test, class_predictions)
print("Final Model Accuracy: {:.2f}%".format(final_accuracy * 100))

from sklearn.metrics import confusion_matrix
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, class_predictions)

# Calculate accuracy for each class
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
sleep_stages = ['Wake', 'N1', 'N2', 'N3', 'REM']

# Print accuracies
for stage, accuracy in zip(sleep_stages, class_accuracies):
    print(f"Accuracy for {stage}: {accuracy * 100:.2f}%")

Final Model Accuracy: 94.96%
Accuracy for Wake: 99.10%
Accuracy for N1: 30.77%
Accuracy for N2: 87.65%
Accuracy for N3: 92.96%
Accuracy for REM: 91.23%


In [144]:
from sklearn.metrics import cohen_kappa_score
import numpy as np

# Compute Cohen's Kappa statistic
kappa_value = cohen_kappa_score(y_test, class_predictions)

# Output the Kappa value
print("Cohen's Kappa statistic:", kappa_value)

# Identify unique sleep stages
unique_stages = np.unique(y_test)

# Dictionary to hold Kappa statistics for each stage
kappa_statistics = {}

# Compute Cohen's Kappa for each stage treated as binary classification
for stage in unique_stages:
    # Create binary labels for the current stage
    true_binary = (y_test == stage).astype(int)
    predicted_binary = (class_predictions == stage).astype(int)
    
    # Calculate the Kappa statistic
    kappa = cohen_kappa_score(true_binary, predicted_binary)
    kappa_statistics[stage] = kappa

# Output the Kappa values for each sleep stage
for stage, kappa in kappa_statistics.items():
    print(f"Cohen's Kappa for sleep stage {stage}: {kappa}")

Cohen's Kappa statistic: 0.8921240972887652
Cohen's Kappa for sleep stage 0: 0.9733207259566594
Cohen's Kappa for sleep stage 1: 0.3794225387363199
Cohen's Kappa for sleep stage 2: 0.8815980465305568
Cohen's Kappa for sleep stage 3: 0.8586146569040368
Cohen's Kappa for sleep stage 4: 0.8218747813377079


In [145]:
from sklearn.metrics import classification_report
report = classification_report(y_test, class_predictions, target_names=['Wake', 'N1', 'N2', 'N3', 'REM'])
print(report)

              precision    recall  f1-score   support

        Wake       0.99      0.99      0.99       775
          N1       0.53      0.31      0.39        26
          N2       0.92      0.88      0.90       162
          N3       0.81      0.93      0.87        71
         REM       0.76      0.91      0.83        57

    accuracy                           0.95      1091
   macro avg       0.81      0.80      0.80      1091
weighted avg       0.95      0.95      0.95      1091

